In [1]:
import os
import yaml
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# get settings from yaml file, make directories (if necessary)

data = yaml.load(open('./settings.yaml', 'r'), yaml.Loader)

images_path = data['images_path']
masks_path = data['masks_path']
image_patches_path = data['image_patches_path']
mask_patches_path = data['mask_patches_path']

patch_size = data['patch_size']
sigma = data['sigma']
num_neg_samples = data['num_neg_samples']

for path in [masks_path, image_patches_path, mask_patches_path]:
    if (os.path.exists(path) == False):
        os.mkdir(path)

In [3]:
# for each image, get info about the image's patches
json_path = os.path.join(images_path, 'meron5_10-50-10.json')
json_data = json.load(open(json_path, 'r'))['_via_img_metadata']
image_data = []

for key in json_data.keys():
    image = json_data[key]
    filename = image['filename']
    filepath = os.path.join(images_path, filename)
    
    region_list = []
    for region in image['regions']:
        try:
            if (region['region_attributes']['Type'] != 'B'):
                continue
        except KeyError:    # some of the patches don't seem to be correctly annotated
            continue
        shape_data = region['shape_attributes']
        temp = {'x': shape_data['x'], 'y': shape_data['y'], 'width': shape_data['width'], 'height': shape_data['height']}
        region_list.append(temp)
    
    image_data.append({'filename': filename, 'filepath': filepath, 'region_list': region_list})

In [4]:
# define a function to draw a 2D gaussian
def draw_2d_gaussian(array, array_point, sigma, dim):
    gaussian_temp = np.zeros([dim, dim])
    for y in range(dim):
        for x in range(dim):
            temp_y, temp_x = y - dim // 2, x - dim // 2
            gaussian_temp[y, x] = (1/(2 * np.pi * sigma ** 2)) * np.exp(-1 * (temp_x ** 2 + temp_y ** 2) / (2 * sigma ** 2))
    gaussian_temp /= gaussian_temp.max()
                                    
    for y in range(array_point[0] - dim // 2, array_point[0] + dim // 2 + 1):
        for x in range(array_point[1] - dim // 2, array_point[1] + dim // 2 + 1):
            if y >= 0 and x >= 0 and y < array.shape[0] and x < array.shape[1]:
                array[y, x] = np.maximum(array[y, x], gaussian_temp[y - (array_point[0] - dim // 2), x - (array_point[1] - dim // 2)])

In [5]:
# create masks w/ gaussian blur at central areas
for image in image_data:
    print("Image:", image['filename'])
    temp = np.zeros(plt.imread(image['filepath']).shape)
    for index in tqdm(range(len(image['region_list']))):
        center = [int(image['region_list'][index]['y'] + 0.5 * image['region_list'][index]['height']),
                  int(image['region_list'][index]['x'] + 0.5 * image['region_list'][index]['width'])]
        draw_2d_gaussian(temp, center, sigma, sigma * 6 + 1)
    np.save(os.path.join(masks_path, image['filename'] + ".npy"), temp)

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted10.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:00<00:00, 96.67it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted20.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 97.54it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted30.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 98.01it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted40.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 98.32it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted50.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:01<00:00, 98.03it/s]


In [6]:
# create a sample centered around each centerpoint

for image in image_data:
    print("Image:", image['filename'])
    
    image_copy = plt.imread(image['filepath'])
    mask_copy = np.load(os.path.join(masks_path, image['filename'] + ".npy"))
    
    for path in [image_patches_path, mask_patches_path]:
        if (os.path.exists(os.path.join(path, image['filename'])) == False):
            os.mkdir(os.path.join(path, image['filename']))
    
    for index in tqdm(range(len(image['region_list']))):                        
        temp = plt.imread(image['filepath']).shape
        center = [int(image['region_list'][index]['y'] + 0.5 * image['region_list'][index]['height']),
                  int(image['region_list'][index]['x'] + 0.5 * image['region_list'][index]['width'])]
        y = np.clip(center[0] - patch_size / 2, 0, temp[0] - patch_size).astype(np.uint16)
        x = np.clip(center[1] - patch_size / 2, 0, temp[1] - patch_size).astype(np.uint16)
        
        image_patch = image_copy[y:y+patch_size, x:x+patch_size]
        mask_patch = mask_copy[y:y+patch_size, x:x+patch_size]
        
        np.save(os.path.join(image_patches_path, image['filename'], "sample_{:03}".format(index)), image_patch)
        np.save(os.path.join(mask_patches_path, image['filename'], "sample_{:03}".format(index)), mask_patch)

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted10.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:01<00:00, 42.77it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted20.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:02<00:00, 40.69it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted30.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:01<00:00, 40.10it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted40.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:01<00:00, 41.37it/s]


Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted50.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:02<00:00, 42.30it/s]


In [7]:
# create a number of "negative samples"
# a "negative sample" is described as having a minimum 

for image in image_data:
    print("Image:", image['filename'])
    
    image_copy = plt.imread(image['filepath'])
    mask_copy = np.load(os.path.join(masks_path, image['filename'] + ".npy"))
    
    counter = 0
    pbar = tqdm(total=num_neg_samples)
    while (counter < num_neg_samples):
        y = np.random.randint(image_copy.shape[0] - patch_size)
        x = np.random.randint(image_copy.shape[1] - patch_size)
        
        if (np.max(mask_copy[int(y + 0.25 * patch_size):int(y + 0.75 * patch_size), int(x + 0.25 * patch_size):int(x + 0.75 * patch_size)]) < 0.8):
            image_patch = image_copy[y:y+patch_size, x:x+patch_size]
            mask_patch = mask_copy[y:y+patch_size, x:x+patch_size]
            
            np.save(os.path.join(image_patches_path, image['filename'], "negsample_{:03}".format(counter)), image_patch)
            np.save(os.path.join(mask_patches_path, image['filename'], "negsample_{:03}".format(counter)), mask_patch)
            
            pbar.update(1)
            counter += 1

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted10.png


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 739/750 [00:44<00:00, 163.81it/s]

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted20.png



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [00:44<00:00, 16.98it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 741/750 [00:54<00:00, 199.22it/s]

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted30.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 748/750 [00:03<00:00, 259.07it/s]

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted40.png



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [00:03<00:00, 199.18it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 743/750 [05:00<00:00, 169.32it/s]

Image: Bubbles_movie_01_x1987x2020x81_3cv2_NLM_template20_search62_inverted50.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [06:21<00:00,  1.77it/s]